# 1vs1

모든 message와 variable belief는 $N(0, 0)$으로 초기화 하고 시작한다.  Prior factor에서 Skill variable로 내려오는 message만 플레이어들의 스킬 분포 초기값 혹은 이전 값이 할당된다.

In [110]:
import trueskill as ts

def print_rating(r):
    print('N({}, {}^2)'.format(round(r.mu,3), round(r.sigma, 3)))
    print('X({}, {})'.format(round(r.pi,3), round(r.tau, 3)))
    
init_mu = 25
init_sigma = init_mu / 3
init_beta = init_sigma / 2
init_tau = init_sigma / 100

env = ts.TrueSkill(mu=init_mu, sigma=init_sigma, beta=init_beta, tau=init_tau, draw_probability=10, backend='mpmath')

p1 = env.Rating(init_mu + 10, 10)
p2 = env.Rating(init_mu, 15)
env.quality(rating_groups=[[p1], [p2]])

0.2703677370942563

In [113]:
init_beta

4.166666666666667

In [46]:
print_rating(p1)

N(35.0, 10.0^2)
X(0.01, 0.35)


In [47]:
print_rating(p2)

N(25.0, 15.0^2)
X(0.004, 0.111)


In [59]:
import math
class Gaussian(object):
    """A model for the normal distribution."""

    #: Precision, the inverse of the variance.
    pi = 0
    #: Precision adjusted mean, the precision multiplied by the mean.
    tau = 0

    def __init__(self, mu=None, sigma=None, pi=0, tau=0):
        if mu is not None:
            if sigma is None:
                raise TypeError('sigma argument is needed')
            elif sigma == 0:
                raise ValueError('sigma**2 should be greater than 0')
            pi = sigma ** -2
            tau = pi * mu
        self.pi = pi
        self.tau = tau

    @property
    def mu(self):
        """A property which returns the mean."""
        return self.pi and self.tau / self.pi

    @property
    def sigma(self):
        """A property which returns the the square root of the variance."""
        return math.sqrt(1 / self.pi) if self.pi else inf

    def __repr__(self):
        return 'N(mu=%.3f, sigma=%.3f)' % (self.mu, self.sigma)

    def _repr_latex_(self):
        latex = r'\mathcal{ N }( %.3f, %.3f^2 )' % (self.mu, self.sigma)
        return '$%s$' % latex

### Factor - Prior - f1

$N(s; \mu, \sigma^2)$

In [79]:
f11 = Gaussian(init_mu + 10, 10); f11

N(mu=35.000, sigma=10.000)

In [80]:
f12 = Gaussian(init_mu, 15); f12

N(mu=25.000, sigma=15.000)

### Message - from Prior to Skill - m1, m2

In [81]:
m1 = f11; m1

N(mu=35.000, sigma=10.000)

In [82]:
m2 = f12; m2

N(mu=25.000, sigma=15.000)

### Variable - Skill - s1, s2

사실 $p(s_1) = m_{1} \cdot m_{3}$인데 $m_3$의 초기값이 $N(0, \inf) = X(0, 0)$이므로, $p(p_1) = m_{5}$ 이다.

In [64]:
s1 = m1; s1

N(mu=35.000, sigma=10.000)

In [65]:
s2 = m2; s2

N(mu=25.000, sigma=15.000)

### Message - from Skill to Likelihood - m3, m4

In [66]:
m3 = m1; m3

N(mu=35.000, sigma=10.000)

In [67]:
m4 = m2; m4

N(mu=25.000, sigma=15.000)

### Factor - Likelihood - f2

$N(p;s, \beta^2)$

### Message - from Likelihood to Performance m5, m6

Table1의 두번째 규칙에 따라 $m_5$과 $m_6$를 결정한다. 해당 공식의 $y$가 $s$에 해당하고 $x$가 $p$에 해당한다.
- $\pi_{f \rightarrow p}^{new} \leftarrow a \cdot (\pi_s - \pi_{f \rightarrow s})$
- $\tau_{f \rightarrow p}^{new} \leftarrow a \cdot (\tau_s - \tau_{f \rightarrow s})$
- $a := (1 + c^2(\pi_s - \pi_{f \rightarrow s}))^{-1}$


$c$값은 공통 분산 $\beta$이고, $m_{f \rightarrow s}$는 $N(0, \inf)$로 초기화되었음
- $c = \beta$
- $\pi_{f \rightarrow s} = 0$
- $\tau_{f \rightarrow s} = 0$


In [70]:
def get_a(pi_y, pi_fy, c):
    return 1.0 / (1.0 + c**2 * (pi_y - pi_fy) )

In [71]:
a1 = get_a(s1.pi, 0, init_beta); a1

0.8520710059171597

In [72]:
a2 = get_a(s2.pi, 0, init_beta); a2

0.9283667621776504

In [73]:
m5 = Gaussian(pi=a1*(s1.pi-0), tau=a1*(s1.tau-0)); m5

N(mu=35.000, sigma=10.833)

In [75]:
m6 = Gaussian(pi=a2*(s2.pi-0), tau=a2*(s2.tau-0)); m6

N(mu=25.000, sigma=15.568)

### Variable - Performance - p1, p2

사실 $p(p_1) = m_{5} \cdot m_{7}$인데 $m_7$의 초기값이 $N(0, \inf) = X(0, 0)$이므로, $p(p_1) = m_{5}$ 이다.

In [76]:
p1 = m5; p1

N(mu=35.000, sigma=10.833)

In [77]:
p2 = m6; p2

N(mu=25.000, sigma=15.568)

### Message - from Performance to SumFactor - m7, m8

In [83]:
m7 = p1; m7

N(mu=35.000, sigma=10.833)

In [84]:
m8 = p2; m8

N(mu=25.000, sigma=15.568)

### Factor - Sum - f3

Table1의 3번째 규칙에 따라 계산된다. $y_1, \cdots, y_n$은 특정 팀에 속하는 플레이어들의 퍼포먼스 $p_1, \cdots, p_n$이며 $x$는 Team performance $t$에 해당한다. 또한 $a_j$는 partial play를 표현하는 각 플레이어의 가중치이다. 1vs1이므로 1인팀이고 팀원의 가중치 $a_1=1$로 둔다. 결국 팀원이 1명이면 SumFactor를 거치면서 변하는 것은 없다.

$\pi_{f \rightarrow t}^{new} \leftarrow \left( \frac{1}{\pi_{p_1} - \pi_{f \rightarrow p_1}} \right)^{-1} = \pi_{p_1} - \pi_{f \rightarrow p_1} = \pi_{p_1} ~ (\because ~ \pi_{f \rightarrow p_1} = 0)$

$\tau_{f \rightarrow t} \leftarrow \pi_{f \rightarrow t}^{new} \cdot \left( \frac{\tau_{p_1} - \tau_{f \rightarrow p_1}}{\pi_{p_1} - \pi_{f \rightarrow p_1}} \right) = \pi_{f \rightarrow t}^{new} \cdot \frac{\tau_{p_1}}{\pi_{p_1}} = \tau_{p_1}$

### Message - from Sum factor to Team performance variable $t$ - m9, m10

In [91]:
m9 = m7; m9

N(mu=35.000, sigma=10.833)

In [92]:
m10 = m8; m10

N(mu=25.000, sigma=15.568)

### Factor - Diff factor - f4

Sum factor와 마찬가지로 Table1의 3번째 규칙에 따라 계산된다. 다만 합이 아니라 첫번째 팀과 두번째 팀 퍼포먼스의 차를 구해야 하므로 가중치로 [1, -1]을 사용한다. (각 팀은 경기 순위로 정렬되어 있다고 가정한다.)

$\pi_{f \rightarrow d}^{new} \leftarrow \left( \frac{1}{\pi_{t_1} - \pi_{f \rightarrow t_2}} - \frac{1}{\pi_{t_2} - \pi_{f \rightarrow t_2}} \right)^{-1} = \left( \frac{1}{\pi_{t_1}} - \frac{1}{\pi_{t_2}} \right)^{-1}$

$\tau_{f \rightarrow d}^{new} \leftarrow \pi_{f \rightarrow d}^{new} \cdot \left( \frac{\tau_{t_1} - \tau_{f \rightarrow t_1}}{\pi_{t_1} - \pi_{f \rightarrow t_1}} - \frac{\tau_{t_2} - \tau_{f \rightarrow t_2}}{\pi_{t_2} - \pi_{f \rightarrow t_2}} \right) = \pi_{f \rightarrow d}^{new} \cdot \left( \frac{\tau_{t_1}}{\pi_{t_1}} - \frac{\tau_{t_2}}{\pi_{t_2}} \right)$

### Message - from Diff factor to Diff variable - m11

In [104]:
pi_new_f_d = 1.0 / (1.0 / m9.pi + 1.0 / m10.pi)
tau_new_f = pi_new_f_d * (m9.tau / m9.pi - m10.tau / m10.pi)
pi_new_f_d
m11 = Gaussian(pi=pi_new_f_d, tau=tau_new_f); m11

N(mu=10.000, sigma=18.966)

In [105]:
print(m11.pi, m11.tau)

0.0027799227799227793 0.027799227799227794


### Variable - Team performance difference - d

In [111]:
m12 = m11; m12

N(mu=10.000, sigma=18.966)

In [108]:
import mpmath
1 - mpmath.ncdf((0 - m12.mu) / m12.sigma)

mpf('0.70098991082468998')

In [109]:
mpmath.npdf((0 - m12.mu) / m12.sigma)

mpf('0.34717209457258763')

In [ ]:
0.2703677370942563

### Win probability for team A wins team B

- https://github.com/sublee/trueskill/issues/1
- Team $a$가 $p$명으로 구성되어 있고 팀원의 스킬이 $N(s_{a1};\mu_{a1}, \sigma_{a1}),~N(s_{a2};\mu_{a2}, \sigma_{a2}), \cdots, N(s_{ap};\mu_{ap}, \sigma_{ap})$이고         
Team $b$가 $q$명으로 구성되어 있고 팀원의 스킬이 $N(s_{b1};\mu_{b1}, \sigma_{b1}),~N(s_{b2};\mu_{b2}, \sigma_{b2}), \cdots, N(s_{bq};\mu_{bq}, \sigma_{bq})$일 때        
Team $a$가 Team $b$를 이길 확률은 아래와 같다.
$$P(p_a > p_b |~ \mathbf{s}_a, \mathbf{s}_b) = \Phi \left(\frac{\sum_i^p \mu_{ai} - \sum_j^q \mu_{bj} - \epsilon}{\sqrt{\beta^2(p+q) + \sum_i^p \sigma_{ai}^2 + \sum_j^q \sigma_{bj}^2}} \right)$$

In [114]:
win_probability([p1], [p2], init_beta)

mpf('0.70098991082468998')

In [107]:
import trueskill as ts
import math
import mpmath

def win_probability(team_a_ratings, team_b_ratings, beta=8.33, epsilon=0):
    """
    
    :param a: Rating list for team a 
    :param b: Rating list for team b
    :param beta: skill chain length
    :param epsilon: draw probability
    :return: win probability for team a
    """
    delta_mu = sum([x.mu for x in team_a_ratings]) - sum([x.mu for x in team_b_ratings]) - epsilon
    sum_sigma = sum([x.sigma ** 2 for x in team_a_ratings]) + sum([x.sigma ** 2 for x in team_b_ratings])
    num_team_members = len(team_a_ratings) + len(team_a_ratings)
    denom = math.sqrt(beta * beta * num_team_members + sum_sigma)
    return mpmath.ncdf(delta_mu / denom)